In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1187447,2021-04-16,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1187448,2021-04-17,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1187449,2021-04-18,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1187450,2021-04-19,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
31803,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
31805,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
31807,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
31809,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
31811,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
731571,2021-04-16,Arkansas,Arkansas,5001,2086,36.00,5000,Arkansas,AR,Arkansas,State,3017804
731573,2021-04-17,Arkansas,Arkansas,5001,2086,36.00,5000,Arkansas,AR,Arkansas,State,3017804
731575,2021-04-18,Arkansas,Arkansas,5001,2086,36.00,5000,Arkansas,AR,Arkansas,State,3017804
731577,2021-04-19,Arkansas,Arkansas,5001,2087,36.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1187447,2021-04-16,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1187448,2021-04-17,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1187449,2021-04-18,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1187450,2021-04-19,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-04-20') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
455,2021-04-20,Snohomish,Washington,34523,563.00,53061,WA,County,822083,4199.45,68.48
908,2021-04-20,Cook,Illinois,523598,10394.00,17031,IL,County,5150233,10166.49,201.82
1360,2021-04-20,Orange,California,269176,4900.00,6059,CA,County,3175692,8476.14,154.30
1811,2021-04-20,Maricopa,Arizona,532133,9787.00,4013,AZ,County,4485414,11863.63,218.20
2262,2021-04-20,Los Angeles,California,1230303,23668.00,6037,CA,County,10039107,12255.10,235.76
...,...,...,...,...,...,...,...,...,...,...,...
1186815,2021-04-20,Wheeler,Oregon,26,1.00,41069,OR,County,1332,1951.95,75.08
1187005,2021-04-20,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00
1187164,2021-04-20,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00
1187319,2021-04-20,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
455,2021-04-20,Snohomish,Washington,34523,563.00,53061,WA,County,822083,4199.45,68.48,68.48,4199.45
908,2021-04-20,Cook,Illinois,523598,10394.00,17031,IL,County,5150233,10166.49,201.82,201.82,10166.49
1360,2021-04-20,Orange,California,269176,4900.00,6059,CA,County,3175692,8476.14,154.30,154.30,8476.14
1811,2021-04-20,Maricopa,Arizona,532133,9787.00,4013,AZ,County,4485414,11863.63,218.20,218.20,11863.63
2262,2021-04-20,Los Angeles,California,1230303,23668.00,6037,CA,County,10039107,12255.10,235.76,235.76,12255.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1186815,2021-04-20,Wheeler,Oregon,26,1.00,41069,OR,County,1332,1951.95,75.08,75.08,1951.95
1187005,2021-04-20,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00,0.00,4044.12
1187164,2021-04-20,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00,0.00,4352.81
1187319,2021-04-20,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00,0.00,591.72


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
764944,2021-04-20,Hale,Alabama,2187,75.00,1065,AL,County,14651,14927.31,511.91,511.91,14927.31,1
832589,2021-04-20,Clarke,Alabama,3474,60.00,1025,AL,County,23622,14706.63,254.00,254.00,14706.63,2
598581,2021-04-20,Lowndes,Alabama,1376,53.00,1085,AL,County,9726,14147.65,544.93,544.93,14147.65,3
445416,2021-04-20,Franklin,Alabama,4214,82.00,1059,AL,County,31362,13436.64,261.46,261.46,13436.64,4
530771,2021-04-20,Etowah,Alabama,13707,348.00,1055,AL,County,102268,13403.02,340.28,340.28,13403.02,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1126194,2021-04-20,Platte,Wyoming,618,11.00,56031,WY,County,8393,7363.28,131.06,131.06,7363.28,19
730026,2021-04-20,Converse,Wyoming,990,17.00,56009,WY,County,13822,7162.49,122.99,122.99,7162.49,20
907471,2021-04-20,Lincoln,Wyoming,1387,12.00,56023,WY,County,19830,6994.45,60.51,60.51,6994.45,21
1009887,2021-04-20,Niobrara,Wyoming,150,2.00,56027,WY,County,2356,6366.72,84.89,84.89,6366.72,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
598581,2021-04-20,Lowndes,Alabama,1376,53.00,1085,AL,County,9726,14147.65,544.93,544.93,14147.65,3,1
764944,2021-04-20,Hale,Alabama,2187,75.00,1065,AL,County,14651,14927.31,511.91,511.91,14927.31,1,2
254660,2021-04-20,Walker,Alabama,7149,277.00,1127,AL,County,63521,11254.55,436.08,436.08,11254.55,28,3
529203,2021-04-20,Clay,Alabama,1505,56.00,1027,AL,County,13235,11371.36,423.12,423.12,11371.36,23,4
645498,2021-04-20,Greene,Alabama,914,34.00,1063,AL,County,8111,11268.65,419.18,419.18,11268.65,27,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
643937,2021-04-20,Hot Springs,Wyoming,364,3.00,56017,WY,County,4413,8248.36,67.98,67.98,8248.36,17,19
907471,2021-04-20,Lincoln,Wyoming,1387,12.00,56023,WY,County,19830,6994.45,60.51,60.51,6994.45,21,20
884874,2021-04-20,Uinta,Wyoming,2155,12.00,56041,WY,County,20226,10654.60,59.33,59.33,10654.60,5,21
253466,2021-04-20,Teton,Wyoming,3707,9.00,56039,WY,County,23464,15798.67,38.36,38.36,15798.67,1,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,68.48,4199.45,19,23
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,68.48,4199.45,19,23
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,68.48,4199.45,19,23
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,68.48,4199.45,19,23
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,68.48,4199.45,19,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1184661,2021-04-16,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1184662,2021-04-17,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1184663,2021-04-18,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1184664,2021-04-19,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
761772,2020-03-30,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,511.91,14927.31,2,1,1.00
761773,2020-03-31,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,511.91,14927.31,2,1,0.00
761774,2020-04-01,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,511.91,14927.31,2,1,0.00
761775,2020-04-02,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,511.91,14927.31,2,1,0.00
761776,2020-04-03,Hale,Alabama,3,0.00,1065,AL,County,14651,20.48,0.00,511.91,14927.31,2,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
922635,2021-04-16,Crook,Wyoming,428,12.00,56011,WY,County,7584,5643.46,158.23,158.23,5643.46,6,23,0.00
922636,2021-04-17,Crook,Wyoming,428,12.00,56011,WY,County,7584,5643.46,158.23,158.23,5643.46,6,23,0.00
922637,2021-04-18,Crook,Wyoming,428,12.00,56011,WY,County,7584,5643.46,158.23,158.23,5643.46,6,23,0.00
922638,2021-04-19,Crook,Wyoming,428,12.00,56011,WY,County,7584,5643.46,158.23,158.23,5643.46,6,23,0.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
596183,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14147.65,1,3,1.00,0.00
596184,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14147.65,1,3,0.00,0.00
596185,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14147.65,1,3,0.00,0.00
596186,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14147.65,1,3,0.00,0.00
596187,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,14147.65,1,3,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
594223,2021-04-16,Albany,Wyoming,4119,11.00,56001,WY,County,38880,10594.14,28.29,28.29,10668.72,23,4,14.00,0.00
594224,2021-04-17,Albany,Wyoming,4119,11.00,56001,WY,County,38880,10594.14,28.29,28.29,10668.72,23,4,0.00,0.00
594225,2021-04-18,Albany,Wyoming,4119,11.00,56001,WY,County,38880,10594.14,28.29,28.29,10668.72,23,4,0.00,0.00
594226,2021-04-19,Albany,Wyoming,4137,11.00,56001,WY,County,38880,10640.43,28.29,28.29,10668.72,23,4,18.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-04-20') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
303980,2021-04-20,Imperial,California,27995,718.00,6025,CA,County,181215,15448.50,396.21,396.21,15448.50,1,2,4.00,0.00
2262,2021-04-20,Los Angeles,California,1230303,23668.00,6037,CA,County,10039107,12255.10,235.76,235.76,12255.10,2,5,330.00,20.00
151402,2021-04-20,San Bernardino,California,294973,4561.00,6071,CA,County,2180085,13530.34,209.21,209.21,13530.34,3,4,90.00,5.00
532291,2021-04-20,Inyo,California,1392,36.00,6027,CA,County,18039,7716.61,199.57,199.57,7716.61,4,25,0.00,0.00
67721,2021-04-20,Stanislaus,California,60583,1032.00,6099,CA,County,550660,11001.89,187.41,187.41,11001.89,5,9,70.00,2.00
31392,2021-04-20,Riverside,California,297620,4547.00,6065,CA,County,2470546,12046.73,184.05,184.05,12046.73,6,6,301.00,19.00
83958,2021-04-20,Tulare,California,49443,831.00,6107,CA,County,466195,10605.65,178.25,178.25,10605.65,7,10,29.00,0.00
53885,2021-04-20,San Joaquin,California,71735,1334.00,6077,CA,County,762148,9412.21,175.03,175.03,9412.21,8,19,208.00,0.00
30572,2021-04-20,Fresno,California,100702,1646.00,6019,CA,County,999101,10079.26,164.75,164.75,10079.26,9,14,81.00,0.00
482397,2021-04-20,Merced,California,31289,453.00,6047,CA,County,277680,11268.01,163.14,163.14,11268.01,10,8,22.00,1.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
1125745,2021-04-20,Lassen,California,5673,1.00,6035,CA,County,30573,18555.59,3.27,3.27,18555.59,56,1,4.00,0.00
303980,2021-04-20,Imperial,California,27995,718.00,6025,CA,County,181215,15448.50,396.21,396.21,15448.50,1,2,4.00,0.00
645440,2021-04-20,Kings,California,22781,245.00,6031,CA,County,152940,14895.38,160.19,160.19,14895.38,11,3,4.00,0.00
151402,2021-04-20,San Bernardino,California,294973,4561.00,6071,CA,County,2180085,13530.34,209.21,209.21,13530.34,3,4,90.00,5.00
2262,2021-04-20,Los Angeles,California,1230303,23668.00,6037,CA,County,10039107,12255.10,235.76,235.76,12255.10,2,5,330.00,20.00
31392,2021-04-20,Riverside,California,297620,4547.00,6065,CA,County,2470546,12046.73,184.05,184.05,12046.73,6,6,301.00,19.00
190341,2021-04-20,Kern,California,108013,1331.00,6029,CA,County,900202,11998.75,147.86,147.86,11998.75,14,7,134.00,9.00
482397,2021-04-20,Merced,California,31289,453.00,6047,CA,County,277680,11268.01,163.14,163.14,11268.01,10,8,22.00,1.00
67721,2021-04-20,Stanislaus,California,60583,1032.00,6099,CA,County,550660,11001.89,187.41,187.41,11001.89,5,9,70.00,2.00
83958,2021-04-20,Tulare,California,49443,831.00,6107,CA,County,466195,10605.65,178.25,178.25,10605.65,7,10,29.00,0.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
303980,2021-04-20,Imperial,California,27995,718.00,6025,CA,County,181215,15448.50,396.21,396.21,15448.50,1,2,4.00,0.00
2262,2021-04-20,Los Angeles,California,1230303,23668.00,6037,CA,County,10039107,12255.10,235.76,235.76,12255.10,2,5,330.00,20.00
151402,2021-04-20,San Bernardino,California,294973,4561.00,6071,CA,County,2180085,13530.34,209.21,209.21,13530.34,3,4,90.00,5.00
532291,2021-04-20,Inyo,California,1392,36.00,6027,CA,County,18039,7716.61,199.57,199.57,7716.61,4,25,0.00,0.00
67721,2021-04-20,Stanislaus,California,60583,1032.00,6099,CA,County,550660,11001.89,187.41,187.41,11001.89,5,9,70.00,2.00
31392,2021-04-20,Riverside,California,297620,4547.00,6065,CA,County,2470546,12046.73,184.05,184.05,12046.73,6,6,301.00,19.00
83958,2021-04-20,Tulare,California,49443,831.00,6107,CA,County,466195,10605.65,178.25,178.25,10605.65,7,10,29.00,0.00
53885,2021-04-20,San Joaquin,California,71735,1334.00,6077,CA,County,762148,9412.21,175.03,175.03,9412.21,8,19,208.00,0.00
30572,2021-04-20,Fresno,California,100702,1646.00,6019,CA,County,999101,10079.26,164.75,164.75,10079.26,9,14,81.00,0.00
482397,2021-04-20,Merced,California,31289,453.00,6047,CA,County,277680,11268.01,163.14,163.14,11268.01,10,8,22.00,1.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_case_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,396.21,15448.50,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,396.21,15448.50,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,396.21,15448.50,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,396.21,15448.50,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,396.21,15448.50,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
3982,04/16/21,Lassen,5669,1.00,18542.50,3.27,3.27,18555.59,56,1,3.00,0.00
3983,04/17/21,Lassen,5669,1.00,18542.50,3.27,3.27,18555.59,56,1,0.00,0.00
3984,04/18/21,Lassen,5669,1.00,18542.50,3.27,3.27,18555.59,56,1,0.00,0.00
3985,04/19/21,Lassen,5669,1.00,18542.50,3.27,3.27,18555.59,56,1,0.00,0.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_death_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,396.21,15448.50,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,396.21,15448.50,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,396.21,15448.50,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,396.21,15448.50,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,396.21,15448.50,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
4068,04/16/21,Merced,31196,452.00,11234.51,162.78,163.14,11268.01,10,8,45.00,0.00
4069,04/17/21,Merced,31196,452.00,11234.51,162.78,163.14,11268.01,10,8,0.00,0.00
4070,04/18/21,Merced,31196,452.00,11234.51,162.78,163.14,11268.01,10,8,0.00,0.00
4071,04/19/21,Merced,31267,452.00,11260.08,162.78,163.14,11268.01,10,8,71.00,0.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)